# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem

m = Mystem() 
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В исходных данных есть много артефактов и неунифицорованных данных: отрицательный стаж, использование разного регистра в колонке с информацией об образовании заемщика, цели получения кредита тоже сформулированы по-разному -- в таком виде их трудно категоризировать. Также таблица содержит избыточные данные строчных типов, что может значительно увеличивать время обработки.

Таким образом исходная таблица требует следующих изменений:
* обработать пропуски;
* найти и постараться устранить артефакты данных;
* заменить вещественные значения на целочисленные;
* устранить дубликаты;
* привести все строковые записи к единому виду;
* добавить столбцы с id для каждой колонки со строковыми значениями;

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isna().sum()) # пропусков значительное количество (порядка 10%)
# проверим, что пропуски в days_employed и total_income встречаются в одних и тех же строках:
data_nan = data[data['total_income'].isna()] # формируем таблицу из строк, содержащих пропущенные значения в 
                                             # total_income
print('Уникальные значения для days_employed:', data_nan['days_employed'].unique(),
      '\nОбщее количество строк в таблице:', len(data_nan))
# то есть для всех пропущенных значений days_employed соответсвующая ячейка total_income тоже содержит
# пропущенное значение.
# посмотрим, есть в данных записи, для которых total_income и days_employed принимали бы знаение 0
print('Число нулевых записей в колонке total_income:',data[data['total_income'] == 0]['total_income'].count())
print('Число нулевых записей в колонке days_employed:',data[data['days_employed'] == 0]['days_employed'].count())
# поскольку в этих колонках не содержится нулей, заменим несуществующие значения нулями
data['days_employed'] = data['days_employed'].fillna(0)
#**********************************
# замена пропусков в total_income
#**********************************
incm_types = data['income_type'].unique() # получаем список профессий

for value in incm_types:
    median = data[data['income_type'] == value]['total_income'].median()
    data.loc[data['income_type'] == value,'total_income'] = median
    


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Уникальные значения для days_employed: [nan] 
Общее количество строк в таблице: 2174
Число нулевых записей в колонке total_income: 0
Число нулевых записей в колонке days_employed: 0


**Вывод**

Столбцы days_employed и total_income содержат пропуски. При этом все строки с пропущенным значением в ячейке days_employed также имеют пропуски в ячеке total_income. Такое могло случиться, если при заполнении "личного дела клиента" отсутствовала информация о его трудовой деятельности.

### Замена типа данных

In [3]:
#*****************************************************************
# посмотрим, чем обусловлен отрицательный стаж в данных
#*****************************************************************

print('Средний трудовой стаж для положительных значений days_employed (годы):',
      int(data[data['days_employed'] > 0]['days_employed'].mean()/365)) # нереалистичные 
                                                                        # значения для трудового стажа
print('Средний трудовой стаж для отрицательных значений days_employed (годы):',
      int(data[data['days_employed'] < 0]['days_employed'].mean()/365)) 

print('Число клиентов с отрицательным стажем:',data[data['days_employed']< 0]['income_type'].count(),'\n*******')

pos_days = data[data['days_employed'] > 0]
neg_days = data[data['days_employed'] < 0]

# посмотрим также на отметку о занятости:
print('days_employed > 0:\n', pos_days['income_type'].value_counts(),'\n*******')
print('days_employed < 0:\n', neg_days['income_type'].value_counts(),'\n*******')

# подавляющее число клиентов с очень большим трудовым стажем -- пенсионеры и немного безработных;
print('Средний возраст клиентов с очень большим трудовым стажем:',round(pos_days['dob_years'].mean(),1))
print('Средний возраст клиентов с отрицательным трудовым стажем:', round(neg_days['dob_years'].mean(),1))

# посмотрим на метрику (возраст (годы) - стаж (годы)):
print('Метрика для людей с отрицательным стажем:',
      round((neg_days['dob_years'] - neg_days['days_employed']/(365)).mean(),1))
# значения метрики для людей с отрицательным стажем близко к среднему значению возраста  в выборке
# возможно, стаж людей с очень большим стажем  изначально записан не в днях, а секундах?
print('Метрика для людей с очень большим стажем (с новым коэффициентом):',
      round((pos_days['dob_years'] - pos_days['days_employed']/(365*24)).mean(),1))
# посмотрим на среднее значение стажа в этом случае:
print('Средний стаж для людей с очень большим стажем (скорректированный, в годах)',
      round(pos_days['days_employed'].mean()/(24*365), 1))
# кажется, тут проблема преобразования форматов дат: неверно расчитали интервал при переходе от 
# UTC-секунд к дням (отрицательный стаж) и указали неверные единицы измерения (неожиданно большой стаж)

Средний трудовой стаж для положительных значений days_employed (годы): 1000
Средний трудовой стаж для отрицательных значений days_employed (годы): -6
Число клиентов с отрицательным стажем: 15906 
*******
days_employed > 0:
 пенсионер      3443
безработный       2
Name: income_type, dtype: int64 
*******
days_employed < 0:
 сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
в декрете              1
предприниматель        1
Name: income_type, dtype: int64 
*******
Средний возраст клиентов с очень большим трудовым стажем: 59.1
Средний возраст клиентов с отрицательным трудовым стажем: 39.8
Метрика для людей с отрицательным стажем: 46.3
Метрика для людей с очень большим стажем (с новым коэффициентом): 17.5
Средний стаж для людей с очень большим стажем (скорректированный, в годах) 41.7


In [4]:
#*****************************************************************
# изменим тип данных в колонках days_employed и total_income
#*****************************************************************
data.loc[data['days_employed'] > 0, 'days_employed'] = pos_days['days_employed']/(24*3600)
data.loc[data['days_employed'] < 0, 'days_employed'] = abs(neg_days['days_employed'])
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
display(data.head(10))
print(data.info())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,142594,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,142594,дополнительное образование
4,0,3,53,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,172357,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,172357,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,142594,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,142594,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


**Вывод**

Столбец days_employed должен содержать информацию о трудовом стаже выраженном в количестве дней. Однако этот столбец содержит отрицательные значения, а средний трудовой стаж для положительных значений составляет порядка 1000 лет.

Выяснилось, что клиентами с положительным стажем являются в основном пенсионеры. Также анализ разности между возрастом клиента и его трудовым стажем показал, что данные с положительными значениями стажа, скорее всего, имеют неверные единицы измерения -- часы вместо дней. После учета коэффициента перевода секунд в количество дней средний стаж для положительных значений стал равным 42 года. Таким образом некорректные данные в исходной таблице можно объяснить неверным переводом из секунд в количество дней стажа: для отрицательных значений из даты начала работы в секундах вычли дату последнего рабочего дня, а для положительные значения не домножили на соответвующий коээфициент перевода количества часов в дни.

В итоге неверные значения в исходной таблице заменялись с помощью метода .loc, а тип данных в колонках days_employed и total_income -- с помощью метода .astype().

### Обработка дубликатов

In [5]:
# посмотрим, сколько явных дубликатов содержится в таблице
print('Явных дубликатов:', data.duplicated().sum())
# пройдемся по колонкам и заметим,что в столбцах education и purpose есть дубликаты:
# написанные с применением разного регистра уровни образования и 
# различные формулировки одной и той же цели (на покупку 
# автомобиля -- приобретение автомобиля);
print(data['education'].value_counts().head())
print(data['education_id'].value_counts().head())
# сначала разберемся с регистром в колонке education:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())
# теперь суммы значений в столбцах education и education_id совпадают;
# посмотрим, что теперь стало с количеством дубликатов:
print('Явных дубликатов после унификации регистра в education:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

Явных дубликатов: 624
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
Явных дубликатов после унификации регистра в education: 721


**Вывод**

В столбце education содержатся одинаковые значения записанные с применением разного регистра ("CРЕДНЕЕ" и "среднее, например). Если не приводить эти значения к единому виду, то сумма дубликатов исходной таблице составит 54 шт. Однако после замены всех прописных букв на строчные число дубликатов увеличивается на 17 шт.

Дубликаты удалялись методом drop_duplicates() с последующим обновлением индексов таблицы.

### Лемматизация

In [6]:
lemmas_list = []
for i in range(50):
    lemmas_list.append(m.lemmatize(data['purpose'][i]))
#print(sorted(lemmas_list))

# унифицируем данные в столбце purpose:
# добавим дополнительный столбец с кодом цели получения кредита:
def purpose_finder(purpose):
    lem_purpose= m.lemmatize(purpose)
    for word in lem_purpose:
        if word == 'свадьба':
            return 0
        if (word == 'недвижимость') or (word == 'жилье'):
            return 1
        if word == 'автомобиль':
            return 2
        if word == 'образование':
            return 3
    if word not in lem_purpose:
        return -1
   
data['purpose_id'] = data['purpose'].apply(purpose_finder)
replacement = ['свадьба', 'недвижимость', 'автомобиль', 'образование']
for i in range(4):
    data.loc[data['purpose_id'] == i, 'purpose'] = replacement[i]
print(data.groupby('purpose')['purpose'].count().sort_values(ascending=False))
# операции с недвижимостю -- самая частая цель получения кредита

purpose
недвижимость    10506
автомобиль       4193
образование      3915
свадьба          2190
Name: purpose, dtype: int64


**Вывод**

В столбце purpose можно выделить несколько типов целей для получения кредита. Однако цели сформулированы по-разному, что затрудняет анализ. С помощью лемматизации были выделены несколько категорий целей для получения кредитов:
* проведение свадьбы;
* операции с недвижимостью (покупка, ремонт, строительство и т.д.);
* приобретение автомобиля;
* получение образования;

После замены цели кредитования на одну из 4 категорий, каждой категории был присвоен id и записан в столбце purpose_id в исходной таблице. 

Исходя из данных обновленной таблицы самой популярной причниной для оформления заема являются операции с недвижимостью.

### Категоризация данных

In [9]:
# Посмотрим на возможное количество детей:
print(data['children'].value_counts())
# есть странные значения: 20 и -1;
# записи со странным количеством детей составляют меньше 1% от общего числа записей в таблице,
# так что удалим их:
data = data.drop(index=data[data['children'] == 20].index) 
data = data.drop(index=data[data['children'] == -1].index)
def chld(amount):
    if amount == 0:
        return 0
    else:
        return 1
data['children_id'] = data['children'].apply(chld) 

# ****************************************
# категоризируем доход
# ****************************************
print(data['total_income'].describe())
def income_analyzer(income):
    if income <= 110e3:
        return 0
    if (income > 110e3) & (income <= 170e3):
        return 1
    if  (income > 170e3) & (income <= 230e3):
        return 2
    if  (income > 230e3):
        return 3
data['income_id'] = data['total_income'].apply(income_analyzer)

 0     13446
 1      4803
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
count     20681.000000
mean     146753.618684
std       17454.454100
min       53829.000000
25%      142594.000000
50%      142594.000000
75%      150447.000000
max      499163.000000
Name: total_income, dtype: float64


In [10]:
# ****************************************
# создадим словари
# ****************************************
education_dict = data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
display(education_dict)

family_status_dict = data[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(family_status_dict)

purpose_dict = data[['purpose_id','purpose']]
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
display(purpose_dict)

income_dict = data[['income_id']]
income_dict = income_dict.drop_duplicates().reset_index(drop=True)
income_list = ['высокий','выше среднего', 'средний','ниже среднего']                    
income_dict['income'] = income_list
income_dict = income_dict.sort_values(by='income_id').reset_index(drop=True)
display(income_dict)

children_dict = data[['children_id']]
children_dict = children_dict.drop_duplicates().reset_index(drop=True)
children_list = ['есть дети', 'нет детей']
children_dict['children'] = children_list
display(children_dict)

data_small = data[['debt', 'children_id', 'children', 'family_status_id', 'income_id', 'purpose_id']]
#display(data_small.head(10))

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


,purpose_id,purpose
0,1,недвижимость
1,2,автомобиль
2,3,образование
3,0,свадьба


,income_id,income
0,0,ниже среднего
1,1,высокий
2,2,выше среднего
3,3,средний


,children_id,children
0,1,есть дети
1,0,нет детей


**Вывод**

Для удобства дальнейшей работы с данными проводилась категоризация значений исходной таблицы: созданы словари для всех значений имеющих строковый тип, а также для значений суммарного дохода клиента. Таким образом быди выделены следующий словари:

1. словарь для уровня дохода клиента;
2. словарь для семейного статуса клиента;
3. словарь для цели получения кредита;
4. словарь уровня образования клиента;
5. словарь количества детей в семье у клиента;

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
# функция для составления сравнительных таблиц
def percentage_definder(dataset, key, hypothesis, hypothesis_id):
    if hypothesis != 'children':
        tmp = dataset[dataset[hypothesis] == hypothesis_id] # создаем промежуточную таблицу из нужных
                                                            # столбцов исходной
        total_amnt = len(tmp)
        return tmp[tmp['debt'] == key]['debt'].count()/total_amnt # считаем отношение количества должников
                                                                  # к суммарному количеству клиентов в 
                                                                  # выбранной категории
    
    else:
        if hypothesis_id == 0: # объединим всех клиентов, имеющих 1 ребенка и более
                               # в отдельную категорию
            tmp = dataset[dataset[hypothesis] == hypothesis_id]
            total_amnt = len(tmp)
            return tmp[tmp['debt'] == key]['debt'].count()/total_amnt
        else:
            tmp = dataset[dataset[hypothesis] >= hypothesis_id]
            total_amnt = len(tmp)
            return tmp[tmp['debt'] == key]['debt'].count()/total_amnt

In [12]:
debts_data = data_small[['debt', 'children_id']]
debts_amnt = []
for i in range(len(children_dict)):
    debts_amnt.append('{:.01%}'.format(percentage_definder(debts_data, 1, 'children_id', i))) 

hypothesis_chld_data = {'Количество клиентов, имеющих задолженности': debts_amnt}
hypothesis_chld = pd.DataFrame(hypothesis_chld_data, index=['нет детей в семье', 'есть ребенок'])  
display(hypothesis_chld)

,"Количество клиентов, имеющих задолженности"
нет детей в семье,7.9%
есть ребенок,9.2%


In [13]:
# сводная таблица для заависимости между наличием детей и возвратом кредита в срок
chld_pivot = data.pivot_table(index='debt', columns='children_id', values='days_employed', aggfunc='count')
chld_pivot[0] = round(100*chld_pivot[0]/len(data[data['children_id'] == 0]),2)
chld_pivot[1] = round(100*chld_pivot[1]/len(data[data['children_id'] == 1]), 2)
display(chld_pivot)

children_id,0,1
debt,,
0,92.09,90.75
1,7.91,9.25


**Вывод**

Клиентсы с детьми чаще не возвращают кредиты в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
debts_data = data_small[['debt', 'family_status_id']]
debts_amnt = []
for i in range(len(family_status_dict)):
    debts_amnt.append('{:.01%}'.format(percentage_definder(debts_data, 1, 'family_status_id', i)))   

hypothesis_mrg_data = {'Количество клиентов, имеющих задолженности': debts_amnt}
hypothesis_mrg = pd.DataFrame(hypothesis_mrg_data, index=['в браке', 'в гражданском браке','вдовец/вдова',
                                                          'в разводе','не в браке']) 
display(hypothesis_mrg.sort_values(by='Количество клиентов, имеющих задолженности', ascending=False))

,"Количество клиентов, имеющих задолженности"
не в браке,9.8%
в гражданском браке,9.6%
в браке,7.8%
вдовец/вдова,7.1%
в разводе,7.1%


In [15]:
# сводная таблица для заависимости между наличием детей и возвратом кредита в срок
mrg_pivot = data.pivot_table(index='family_status_id', columns='debt', values='days_employed', aggfunc='count')

for v in range(2):
    for i in range(len(family_status_dict)):
         mrg_pivot[v][i] = 100*mrg_pivot[v][i]/len(data[data['family_status_id'] == i])
display(mrg_pivot)

debt,0,1
family_status_id,,
0,92,7
1,90,9
2,92,7
3,92,7
4,90,9


**Вывод**

Клиенты не состоящие (и не состоявшие) в официальном браке реже возвращают кредиты в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
debts_data = data_small[['debt', 'income_id']]
debts_amnt = []

for i in range(len(income_dict)):
    debts_amnt.append('{:.01%}'.format(percentage_definder(debts_data, 1, 'income_id', i)))
hypothesis_incm_data = {'Количество клиентов, имеющих задолженности': debts_amnt}
hypothesis_incm = pd.DataFrame(hypothesis_incm_data, index= ['ниже среднего', 'средний', 'выше среднего', 'высокий'])
display(hypothesis_incm.sort_values(by='Количество клиентов, имеющих задолженности', ascending=False))

,"Количество клиентов, имеющих задолженности"
средний,8.7%
выше среднего,7.4%
ниже среднего,50.0%
высокий,0.0%


In [17]:
# сводная таблица для заависимости между семейным положением и возвратом кредита в срок
incm_pivot = data.pivot_table(index='income_id', columns='debt', values='days_employed', aggfunc='count')
for v in range(2):
    for i in range(len(income_dict)):
        incm_pivot[v][i] = round(100*incm_pivot[v][i]/len(data[data['income_id'] == i]),2)

display(incm_pivot)

debt,0,1
income_id,,
0,50.00,50.00
1,91.32,8.68
2,92.59,7.41
3,100.00,NaN


**Вывод**

Половина клиентов с уровнем дохода ниже среднего (146тыс.) имеет задолженности по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
debts_data = data_small[['debt', 'purpose_id']]
debts_amnt = []
repaids_amnt = []

for i in range(len(purpose_dict)):
    debts_amnt.append('{:.01%}'.format(percentage_definder(debts_data, 1, 'purpose_id', i)))
    repaids_amnt.append('{:.01%}'.format(percentage_definder(debts_data, 0, 'purpose_id', i)))    
    
hypothesis_prps_data = {'Количество клиентов, имеющих задолженности': debts_amnt}
hypothesis_prps = pd.DataFrame(hypothesis_prps_data, index=['свадьба', 'операции с недвижимостью',
                                                            'автомобиль', 'образование'])
display(hypothesis_prps.sort_values(by='Количество клиентов, имеющих задолженности', ascending=False))

,"Количество клиентов, имеющих задолженности"
автомобиль,9.6%
образование,9.5%
свадьба,8.4%
операции с недвижимостью,7.5%


**Вывод**

Клиенты берущие кредит для оплаты образования и покупки автомобиля чаще не погашают кредит в срок.

## Шаг 4. Общий вывод

Изучалось влияние таких факторов как семейное положение клиента и наличие у него детей на сроки погашения кредита. Для анализа использовалась статистика о платёжеспособности клиентов полученная от банка.

В исходной таблице содержались пропуски значений в столбцах days_employed и total_income. Число пропусков (2174) значительное и составляет около 10% от общего числа записей в таблице. Таким образом просто удалить такие строки без потери информации не представляется возможным, поэтому значения в days_employed были заполнены нулями, а d total_income -- медианным значением для соответствующей категории.

Столбец days_empoyed содержал некорректные значения для трудового стажа клиентов в днях: часть значенияй колонки были отрицательными, а средний стаж рассчитанный из положительных значений составлял порядка 1000 лет. Чтобы понять причину возникновения таких артефактов сравнивалось значение метрики (возраст клиента - стаж) для двух групп. Выяснилось, что для исходных значений стажа эта разница оказалась значительной. Однако если предположить, что единицы измерения для положительных значений стажа указаны неверно, и домножить эти значения на соответсвующий переводной коэффициент, значения метрик становятся сравнимыми. При анализе столбца также выявлено, что положительные значения стажа имеют  клиенты, чей тип занятости соответсвует категории "пенсионеры". Таким образом можно предположить, что причина возникновения этих артефактов -- неверный перевод значений даты из формата unix time в количество дней стажа:
* для отрицательных значений неправильно определили временной промежуток, вычтя дату последнего рабочего дня из даты первого рабочего дня;
* при заполнении данных о клиентах с положительным стажем, скорее всего, стаж не указывался, тк значения среднего возраста и стажа совпадают, а в группу попали в основном одни пенсионеры (и несущественное число безработных). А затем при выгрузке были указаны неверные единицы измерения -- дни вместо часов;

Артефакты были устранены домножением положительных значений стажа и на переводной коэффициент, а от отрицательных значений избавились применением к ним функции abs(); тип значений в столбце был изменен с вещественного на целочисленный методом astype().

Для устранения явных дубликатов применялись стандартные методы pandas: isna() и dropna(). Однако чтобы удалить все явные дубликаты сначала пришлось привести значения в столбце education к единому виду: данные в этом столбце были записаны с применением различного регистра ("CРЕДНЕЕ" и "среднее"). Замена прописных букв на строчные проводилась с помощью метода lower().

Проводилась лемматизация целей кредита с помощью функции Mystem из библиотеки pymystem3. Были выявлены четыре леммы:
1. свадьба;
2. недвижимость;
3. автомобиль;
4. образование;


Для удобства дальнейшего анализа значения столбца purpose заменялись на соответсвущую лемму, также в таблицу был добавлен столбец purpose_id содержащий уникальный id для каждой категории цели получения кредита.

Для проверки гипотез о влиянии семейного положения и количества детей у клиента на сроки погашения кредита данные исходной таблицы категоризировались: значениям всех столбцов содержащих строковые значения были присвоены уникальные id. Данные в столбце total_income были также разбиты на три группы: средний доход, низкий и высокий в зависимости от удаленности от медианного значения. В данных по количеству детей были также обнаружены артефакты: -1 ребенок и 20 детей в семье. Их возникновение может быть объяснено невнимательностью оператора при заполнеии формы. Поскольку число артефактов не велико, то эти строки были удалены из датасета методом drop().

Далее были составлены несколько словарей:
1. словарь с данными об образовании клиента;
2. словарь семейного статуса клиента;
3. словарь целей кредита;
4. словарь уровня дохода клиента;
5. словарь количества детей в семье клиента;

С помощью обновленной таблицы с категоризированными данными определялись факторы, которые могут влиять на способность потенциального заёмщика вернуть кредит банку. Выяснилось, что:
1. клиенты с одним и более ребенком чаще имеют задолженности по кредиту;
2. клиенты не состоящие (и не состоявшие) в официальном браке реже возрвращают кредиты в срок;
3. клиенты с уровнем дохода ниже среднего чаще других категорий имеют задолженности по кредиту;
4. клиенты берущие кредит для оплаты образования и покупки автомобиля чаще не погашают кредит в установленные сроки.

Таким образом при построении модели кредитного скоринга стоит учитывать такие факторы как наличие детей в семье клиента, его семейеого положение, цель получения кредита и доход клиента.